In [1]:
from netgen.occ import *

cube1 = Box((-250,-50,5), (250,50,10))
cube2 = Box((-250,-50,-5), (250,50,5))

cube1.faces.Min(Z).name = "bottom"
cube1.faces.Min(X).name = "left1"
cube2.faces.Min(X).name = "left2"
cube1.faces["bottom"].edges.Min(Y).name = "front"
cube1.faces["bottom"].edges.Max(Y).name = "back"
print([s.name for s in cube1.faces.faces])
print([s.name for s in cube2.faces.faces])

cube = Glue([cube1,cube2])

netmesh = OCCGeometry(cube).GenerateMesh(maxh=10) #/1.975

from ngsolve import *
from ngsolve.webgui import Draw

mesh = Mesh(netmesh)

print(mesh.ne)

Draw (mesh)

['left1', None, None, None, 'bottom', None]
['left2', None, None, None, None, None]
6791


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [2]:
# Plate projections
n = specialcf.normal(mesh.dim)

def Anti(vec):
    return CF( (0, -vec[2], vec[1],
                vec[2], 0, -vec[0],
                -vec[1], vec[0], 0), dims = (3,3) ) 

def Dyad(v1, v2):
    return CF( (v1[0]*v2[0], v1[0]*v2[1], v1[0]*v2[2],
                v1[1]*v2[0], v1[1]*v2[1], v1[1]*v2[2],
                v1[2]*v2[0], v1[2]*v2[1], v1[2]*v2[2]), dims = (3,3) ) 

Antin = Anti(n)
Q = Dyad(n, n)
P = Id(3) - Q

# Beam projections
t = CF ( (1, 0, 0), dims = (3,1) )
d2 = CF( (0, 1, 0), dims = (3,1) )
d3 = CF( (0, 0, 1), dims = (3,1) )

T = Dyad(t,t)
O = Id(3) - T

Antid2 = Anti(d2)
Antid3 = Anti(d3)

In [3]:
# Volume coefficients (organic tissue) 
mue = 30 
muc = 1
lame = 98.5 
Lc = 10**-2

def Ce(mat):
    return 2*mue*mat + lame*Trace(mat)*Id(3)


# Shell coefficients (bone)
h = 1.6
mues = 2122.64 
lames = 289.451
mucs = 10000


# Beam coefficients (bone)
Ee = mues * ( 3*lames + 2 *mues ) / (lames + mues)
A = pi * 0.8**2
I = 0.25 * pi * 0.8**4


# Plane stress
lamess = 2*lames*mues / (lames + 2*mues)


def De(mat):
    return 2*mues*mat + lamess*Trace(mat)*P
    

muelca1 = 10867.9
muelca2 = 122264
muelca3 = 0


def Vol(mat):
    return (1/3) * Trace(mat) * Id(3) 


def Dev(mat):
    return mat - Vol(mat)


def L(mat):
    return  muelca1 * Dev(Sym(mat)) + muelca2 * Skew(mat) + muelca3 * Vol(mat)

uD = CF ( (0,0,0), dims = (3,1) )
rD = CF ( (0,0,0), dims = (3,1) )

q = 1e-5 * CF ( (0,0,-1), dims = (3,1) )
m = CF ( (0,0,0), dims = (3,1) )

In [4]:
order = 2
fesH = H1(mesh, order=order, dirichlet="left1|left2")
fesR = H1(mesh, order=order+1)
fes = fesH * fesH * fesH * fesR * fesR * fesR

(u1,u2,u3,r1,r2,r3), (du1,du2,du3,dr1,dr2,dr3) = fes.TnT()

In [5]:
# Volume entities
u = CF( (u1,u2,u3), dims = (3,1) )
du = CF( (du1,du2,du3), dims = (3,1) )

r = CF( (r1,r2,r3), dims = (3,1) )
dr = CF( (dr1,dr2,dr3), dims = (3,1) )

Du = CF ( (grad(u1), grad(u2), grad(u3)), dims = (3,3) ) 
Dr = CF ( (grad(r1), grad(r2), grad(r3)), dims = (3,3) ) 

SymDu = Sym(Du)
SkewDu = Skew(Du)

R = Anti(r)
CurlR = Trace(Dr)*Id(3) - Dr.trans

In [6]:
# Plate entities
Dtu = CF ( (grad(u1).Trace(), grad(u2).Trace(), grad(u3).Trace()), dims = (3,3) )  * P
Dtr = CF ( (grad(r1).Trace(), grad(r2).Trace(), grad(r3).Trace()), dims = (3,3) )  * P 

Dtcovu = P * Dtu
Dtcovr = P * Dtr

SymDtcovu = Sym(Dtcovu)  
SkewDtcovu = Skew(Dtcovu)  

Rt = R * P
Rtcov = P * Rt

CurltR = InnerProduct(Dtr, P)*Id(3) - P * Dtr.trans

In [7]:
# Beam entities
Dttu = CF ( (grad(u1).Trace().Trace(), grad(u2).Trace().Trace(), grad(u3).Trace().Trace()), dims = (3,3) )  * T
Dttr = CF ( (grad(r1).Trace().Trace(), grad(r2).Trace().Trace(), grad(r3).Trace().Trace()), dims = (3,3) )  * T  

Dttcovu = T * Dttu
Dttcovr = T * Dttr

SymDttcovu = Sym(Dttcovu)  

Rtt = R * T
Rttcov = T * Rt

CurlttR = InnerProduct(Dttr, P)*Id(3) - P * Dttr.trans

In [8]:
a = BilinearForm(fes, symmetric=True, symmetric_storage=True, condense=True)

# Volume 
a += (Variation(0.5*(
    InnerProduct(SymDu,Ce(SymDu)) 
    + 2*muc * InnerProduct(SkewDu - R, SkewDu - R)
    + mue*Lc**2 * InnerProduct(CurlR, CurlR)
)*dx))

# Plate
# a += Variation(0.5*(
#     h * InnerProduct(SymDtcovu,De(SymDtcovu)) 
#     + h * 2*mucs * InnerProduct(SkewDtcovu - Rtcov, SkewDtcovu - Rtcov)
#     + h * (mues + mucs) * InnerProduct(Q*(Dtu - Rt), Q*(Dtu - Rt))
#     + (h**3 / 12) * InnerProduct(Sym(Antin * Dtcovr), De(Sym(Antin * Dtcovr)))
#     + (h**3 / 12) * 2*mucs * InnerProduct(Skew(Antin * Dtcovr), Skew(Antin * Dtcovr)) 
#     + h * InnerProduct(CurltR, L(CurltR))
# )*ds("bottom"))

# beam
a += Variation(0.5*(
    Ee*A * InnerProduct(SymDttcovu, SymDttcovu) 
    + Ee*I * InnerProduct(Sym(P * Antid2 * Dttr), Sym(P * Antid2 * Dttr)) 
    + Ee*I * InnerProduct(Sym(P * Antid3 * Dttr), Sym(P * Antid3 * Dttr))
    + (mues + mucs)*A * InnerProduct(O*(Dttr - Rtt), O*(Dttr - Rtt))
    + (mues + mucs)*I * InnerProduct(O*Antid2*Dttr, O*Antid2*Dttr)
    + (mues + mucs)*I * InnerProduct(O*Antid3*Dttr, O*Antid3*Dttr)
    + A * InnerProduct(CurlttR, L(CurlttR))
)*dx(mesh.BBoundaries("front|back")))

f = LinearForm(fes)
f += (InnerProduct(du,q) + InnerProduct(dr,m))*dx

In [9]:
sol = GridFunction(fes)

sol.vec[:] = 0

v = sol.vec.CreateVector()
w = sol.vec.CreateVector()

In [10]:
with TaskManager():
    f.Assemble()
    a.AssembleLinearization(sol.vec)
    inv = a.mat.Inverse(fes.FreeDofs(a.condense), inverse="pardiso")

    a.Apply(sol.vec, v)
    
    v.data -= f.vec
    if a.condense:
        v.data += a.harmonic_extension_trans * v
    w.data = inv * v
    if a.condense:
        w.data += a.harmonic_extension * w
        w.data += a.inner_solve * v
    sol.vec.data -= w

In [11]:
gfu1,gfu2,gfu3, gfr1,gfr2,gfr3 = sol.components

gfu = CF( (gfu1,gfu2,gfu3), dims = (3,1) )

Draw(gfu, mesh, "u", deformation=True)
# Draw(Norm(gfr), mesh, "r")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [12]:
# vtk = VTKOutput(ma=mesh,
#                 coefs=[gfu],
#                 names = ["displacement"],
#                 filename="cantilever-mesh",
#                 subdivision=0)
# # Exporting the results:
# vtk.Do()